### Data Science 100 - Final Project

An investigation of the popularity of certain play-times using data from a MineCraft server. 

By Group #4: Chloe Glesby, Ryann Wilson, 

### Introduction:
MineCraft, published by Mojang Studios is an online video game where users can place ‘blocks’ to create and build different structures. The game has a multiplayer aspect where users can interact and collaborate. The Pacific Laboratory for Artificial Intelligence (PLAI), founded by Professor Frank Wood created a MineCraft server to conduct research and data analysis about this video game and its users. Once players log onto this specific server, all kinds of data is being collected. Some examples of this data include the kind of player they are, the time of day they log on and off, and what things they say while playing. This information can be useful for the PLAI team as it can showcase patterns to help formulate models for prediction purposes. These predictions can be used in many aspects of video game research including what the most popular times will be and how many people the server needs to accommodate - which is the goal of this data analysis. 

Using the PLAI team’s minecraft data, an analysis will be conducted in order to predict:

a) What time windows are most likely to have a large number of simultaneous players?

b) What are the average length of playing sessions during different times of the day? (probably needs to be re-worded)

To begin the data anlysis, some Python packages must first be imported into the notebook:

In [23]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn import set_config


# Simplify working with large datasets in Altair
alt.data_transformers.enable('vegafusion')

# Output dataframes instead of arrays
set_config(transform_output="pandas")

Next, the data was loaded as a simple data frame using the 'pandas' package 

In [32]:
sessions_data = pd.read_excel(
    'https://1drv.ms/x/s!AvNVskqz4Sctgq59DaKuXY5LEOJnyw?e=XZTBpW&nav=MTVfe0IzNEJFRjlFLTE0NDUtNDg5RS04NDY1LTNBNjRBMjdDM0U0OH0'
) # the file path for the dataframe in brackets
sessions_data.head(20)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

### Method of Data Analysis

Clustering!

### The Data 
The Sessions dataframe has 1535 observations and 5 variables which provide information about the time frames that players of the PLAICraft server logged on and off. 

https://drive.google.com/file/d/14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB/edit

- The column `hashedEmail` describes induvidual player email's in an encoded fashion, using a string of numbers and letters. The observations in this column are not useful in this data analysis, as the identify of the players is not important to the type of predictive questions in this analysis.  
- The columns `start_time` and `end_time` provide the exact time and date that players logged on and off the server in day-month-year-time format. This makes the data untidy though, as there are multiple variables within a cell. 
- The columns `original_start_time` and `original_end_time` are information about the log-on and off times in Unix time format. This column is not useful for this data analysis as it is in a time format that is difficult to manipulate and interpret.

Using the `count` - a simple numbered list of players, and the`'start_time` and `end_time` columns, some prelminary data wrangling and filtering will be conducted to better visulize any patterns in the Sessions data set.






The start and end time format is easier to use than Unix time, but it will be more efficient to organize it into a format that Python recognizes and classifies as tidy:

In [4]:
#to convert the time into 'datetime' format and split each variable into a sperate column

sessions_data['start_time'] = pd.to_datetime(sessions_data['start_time'],format='%d/%m/%Y %H:%M')
sessions_data['end_time'] = pd.to_datetime(sessions_data['end_time'],format='%d/%m/%Y %H:%M')
sessions_data["start_hour"] = sessions_data["start_time"].dt.hour
sessions_data["start_month"] = sessions_data["start_time"].dt.month
sessions_data["start_year"] = sessions_data["start_time"].dt.year 

# drop unnecessary columns 
sessions_data = sessions_data.drop(columns = ['hashedEmail','original_start_time','original_end_time'])

# tell python to show the new data set 
sessions_data

,start_time,end_time,start_hour,start_month,start_year
0,2024-06-30 18:12:00,2024-06-30 18:24:00,18,6,2024
1,2024-06-17 23:33:00,2024-06-17 23:46:00,23,6,2024
2,2024-07-25 17:34:00,2024-07-25 17:57:00,17,7,2024
3,2024-07-25 03:22:00,2024-07-25 03:58:00,3,7,2024
4,2024-05-25 16:01:00,2024-05-25 16:12:00,16,5,2024
...,...,...,...,...,...
1530,2024-05-10 23:01:00,2024-05-10 23:07:00,23,5,2024
1531,2024-07-01 04:08:00,2024-07-01 04:19:00,4,7,2024
1532,2024-07-28 15:36:00,2024-07-28 15:57:00,15,7,2024
1533,2024-07-25 06:15:00,2024-07-25 06:22:00,6,7,2024


Next, we created a column in the dataframe that calculates the total minutes played in each playing session. This will help us in determining how long playing sessions are during different time solts in the day.

In [5]:
sessions_data['time_played_mins'] = ((sessions_data['start_time'].dt.hour * 60 + sessions_data['start_time'].dt.minute) -
(sessions_data['end_time'].dt.hour * 60 + sessions_data['end_time'].dt.minute)).abs()
sessions_data

,start_time,end_time,start_hour,start_month,start_year,time_played_mins
0,2024-06-30 18:12:00,2024-06-30 18:24:00,18,6,2024,12.0
1,2024-06-17 23:33:00,2024-06-17 23:46:00,23,6,2024,13.0
2,2024-07-25 17:34:00,2024-07-25 17:57:00,17,7,2024,23.0
3,2024-07-25 03:22:00,2024-07-25 03:58:00,3,7,2024,36.0
4,2024-05-25 16:01:00,2024-05-25 16:12:00,16,5,2024,11.0
...,...,...,...,...,...,...
1530,2024-05-10 23:01:00,2024-05-10 23:07:00,23,5,2024,6.0
1531,2024-07-01 04:08:00,2024-07-01 04:19:00,4,7,2024,11.0
1532,2024-07-28 15:36:00,2024-07-28 15:57:00,15,7,2024,21.0
1533,2024-07-25 06:15:00,2024-07-25 06:22:00,6,7,2024,7.0


To complete our wrangling, we created another dataframe with the start hours and the total count of playing sessions that began in that hour. From this dataframe, will be able to easily visualize the distribution of playing hours, as well as easily perform our data analysis later.

In [6]:
start_hour_count = sessions_data["start_hour"].value_counts()
start_hour_data = pd.DataFrame(start_hour_count).reset_index()
start_hour_data

,start_hour,count
0,2,152
1,4,151
2,3,131
3,0,128
4,23,122
5,21,91
6,22,89
7,5,88
8,1,79
9,20,74


#### Visualizations

Below are vizualizations comparing the relationship between hour of the day and number of playing sessions (`Number of playing sessions vs hour of the day`) and the relationship between hour of the day minutes played per session (`Total minutes per playing session vs hour the day`). In these vizualizations, we can see some obvious clusters, so we know clustering is an effective method of anaylsis to use. In `Number of playing sessions vs hour of the day`, we can see a cluster early in the day with a high number of playing sessions, a cluster mid-day with a low number of playing sessions, and a cluster late in the day with a high number of playing session. In `Total minutes per playing session vs hour the day`, we can see a cluster early in the day with few total minutes, a cluster late in the day with few total minutes, and a cluster late in the day with high total minutes.

In [7]:
hours_scatter = alt.Chart(start_hour_data).mark_point().encode(
    x=alt.X('start_hour').title('Hour of the day'),
    y=alt.Y('count').title('Number of players'),
).properties(title='Number of playing sessions vs hour of the day')
hours_scatter

alt.Chart(...)

In [8]:
minutes_played_scatter = alt.Chart(sessions_data).mark_point().encode(
    x=alt.X("start_hour").title("Hour of the day"),
    y=alt.Y("time_played_mins").title("Total minutes per playing session")
).properties(title="Total minutes per playing session vs hour the day")
minutes_played_scatter

alt.Chart(...)

#### Data Summary

For our two questions, we are most interested in the number of players who began their playing session each hour, as well as the relationship between the length in minutes of each playing session and the hour in which the sessions began. For our first question, we can see from our `start_hour_data` dataframe and `Number of playing sessions vs hour of the day` visualization that 2 AM, 4 AM, and 3 AM are the most popular times for players to begin playing and 10 AM, 1 PM, and 12 PM are the least popular times for players to begin playing. For our second question, we can most clearly see the relationship between the variables in our vizualization, `Total minutes per playing session vs hour the day`. The most common time for a playing session to be long (> 1000 minutes) is late in the day, at hours between 8 PM and 11 PM.

In [9]:
time_played_plot = alt.Chart(sessions_data).mark_bar().encode(
    x=alt.X('time_played_mins').title('Time played (min)').bin(maxbins=20),
    y=alt.Y('count()').title('Number of players')
).properties(title='Distribution of player count based on time played')
time_played_plot 
# shows how long players typically play for

alt.Chart(...)

In [10]:
hours_plot = alt.Chart(sessions_data).mark_bar().encode(
    x=alt.X('start_hour').title('Hour of the day').bin(maxbins=20),
    y=alt.Y('count()').title('Number of players')
).properties(title='Distribution of player count throughout the day')
hours_plot

alt.Chart(...)

In [11]:
hours_scatter = alt.Chart(sessions_data).mark_point().encode(
    x=alt.X('start_hour').title('Hour of the day'),
    y=alt.Y('count()').title('Number of players')
).properties(title='Distribution of player count throughout the day')
hours_scatter

alt.Chart(...)

In [12]:
hours_line = alt.Chart(sessions_data).mark_line().encode(
    x=alt.X('start_hour').title('Hour of the day').bin(maxbins=20),
    y=alt.Y('count()').title('Number of players')
).properties(title='Distribution of player count throughout the day')
hours_line

alt.Chart(...)

In [13]:
sessions_data

,start_time,end_time,start_hour,start_month,start_year,time_played_mins
0,2024-06-30 18:12:00,2024-06-30 18:24:00,18,6,2024,12.0
1,2024-06-17 23:33:00,2024-06-17 23:46:00,23,6,2024,13.0
2,2024-07-25 17:34:00,2024-07-25 17:57:00,17,7,2024,23.0
3,2024-07-25 03:22:00,2024-07-25 03:58:00,3,7,2024,36.0
4,2024-05-25 16:01:00,2024-05-25 16:12:00,16,5,2024,11.0
...,...,...,...,...,...,...
1530,2024-05-10 23:01:00,2024-05-10 23:07:00,23,5,2024,6.0
1531,2024-07-01 04:08:00,2024-07-01 04:19:00,4,7,2024,11.0
1532,2024-07-28 15:36:00,2024-07-28 15:57:00,15,7,2024,21.0
1533,2024-07-25 06:15:00,2024-07-25 06:22:00,6,7,2024,7.0


I kept the datetime columns bc I think they might be useful if we want to pull out other values later

In [14]:
import altair as alt
import pandas as pd